###### Sai deepthi(S20180010087)
##### Sowmya(S20180010187)
##### Manjju shree(S20180010055)
##### Swathi(S20180010172)

In [1]:
import random
import numpy as np
from data_process import get_CIFAR10_data, get_MUSHROOM_data
from scipy.spatial import distance
#from models import Perceptron, SVM, Softmax, Logistic
from models import SVM, Softmax
%matplotlib inline

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [2]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [3]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [4]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

# Loading Mushroom

In the following cells we determine the splitting of the mushroom dataset.
<br /> TRAINING + VALIDATION = 0.8, TESTING = 0.2

In [5]:
# TRAINING = 0.6 indicates 60% of the data is used as the training dataset.
VALIDATION = 0.2

In [6]:
data = get_MUSHROOM_data(VALIDATION)
X_train_MR, y_train_MR = data['X_train'], data['y_train']
X_val_MR, y_val_MR = data['X_val'], data['y_val']
X_test_MR, y_test_MR = data['X_test'], data['y_test']
n_class_MR = len(np.unique(y_test_MR))

print("Number of train samples: ", X_train_MR.shape[0])
print("Number of val samples: ", X_val_MR.shape[0])
print("Number of test samples: ", X_test_MR.shape[0])

Number of train samples:  4874
Number of val samples:  1625
Number of test samples:  1625


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [7]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Support Vector Machines (with SGD)

First, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with :
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/SVM.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on CIFAR

In [37]:
lr = 0.01
n_epochs = 500
reg_const = 0.05

svm_CIFAR = SVM(n_class_CIFAR, lr, n_epochs, reg_const)
svm_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

In [38]:
pred_svm = svm_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_CIFAR)))

The training accuracy is given by: 21.625000


### Validate SVM on CIFAR

In [39]:
pred_svm = svm_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_CIFAR)))

The validation accuracy is given by: 21.370000


### Test SVM on CIFAR

In [40]:
pred_svm = svm_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_CIFAR)))

The testing accuracy is given by: 21.700000


## Train SVM on Mushroom

In [12]:
lr = 0.01
n_epochs = 100
reg_const = 0.05

svm_MR = SVM(n_class_MR, lr, n_epochs, reg_const)
svm_MR.train(X_train_MR, y_train_MR)

In [13]:
pred_svm = svm_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_MR)))

The training accuracy is given by: 79.154698


### Validate SVM on Mushroom

In [14]:
pred_svm = svm_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_MR)))

The validation accuracy is given by: 77.046154


## Test SVM on Mushroom

In [15]:
pred_svm = svm_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_MR)))

The testing accuracy is given by: 79.384615


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with :
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/Softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on CIFAR

In [57]:
lr = 0.001
n_epochs = 500
reg_const = 0.05

softmax_CIFAR = Softmax(n_class_CIFAR, lr, n_epochs, reg_const)
softmax_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

In [50]:
pred_softmax = softmax_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_CIFAR)))

The training accuracy is given by: 9.965000


### Validate Softmax on CIFAR

In [51]:
pred_softmax = softmax_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_CIFAR)))

The validation accuracy is given by: 10.140000


### Testing Softmax on CIFAR

In [52]:
pred_softmax = softmax_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_CIFAR)))

The testing accuracy is given by: 10.000000


## Train Softmax on Mushroom

In [53]:
lr = 0.001
n_epochs =  100
reg_const = 0.05

softmax_MR = Softmax(n_class_MR, lr, n_epochs, reg_const)
softmax_MR.train(X_train_MR, y_train_MR)

[0.6838604381027243,
 1.2552586304855704,
 0.8009097033511252,
 1.1641296533016463,
 1.6050499984957556,
 0.6030587799283047,
 0.7714372091274135,
 1.785979598764577,
 1.208621874964911,
 0.30419280642825236,
 0.3032705499411603,
 0.7165906376658655,
 0.961430916439314,
 1.2331360657203951,
 0.2591570707222781,
 0.5880439724807864,
 0.5287377653600317,
 0.15842831040616398,
 0.3394432806691065,
 1.4439649156048124,
 1.1303505658913227,
 2.334386773188463,
 0.8158220185761286,
 0.7051722705110395,
 1.1237820074856395,
 0.27174862642520875,
 0.5724824884996061,
 0.4643843788488118,
 1.2386185135054977,
 3.6384477534196336,
 0.5513038049838774,
 0.1494093128977751,
 0.06388188173343909,
 1.353632939280057,
 2.551437342416409,
 1.7221338117863454,
 0.804202419389961,
 0.9954411263688163,
 1.9157389704538719,
 0.14971887529967667,
 0.8347386365539177,
 1.9297096710566155,
 0.689534240098415,
 0.12618029862037136,
 0.5327036518820618,
 0.5768939577108803,
 1.00796361577615,
 1.78618986430697

In [54]:
pred_softmax = softmax_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_MR)))

The training accuracy is given by: 83.709479


### Validate Softmax on Mushroom

In [55]:
pred_softmax = softmax_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_MR)))

The validation accuracy is given by: 82.030769


### Testing Softmax on Mushroom

In [56]:
pred_softmax = softmax_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_MR)))

The testing accuracy is given by: 83.507692
